In [1]:
# Import all the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [23]:
#================================================================================================
# USER INPUT!
# Here I specify which data files need reading in
datafilenames = ["train.csv"] # In this case I only have one file to read in
#================================================================================================

In [24]:
alldataframes = [pd.read_csv(filename) for filename in datafilenames]

In [4]:
# Here are a bunch of functions that are aimed at detecting dirty data and cleaning it

# This function takes a column and does a tally of the different types of entries (int, float, string, etc.)
# It then spits out a tuple of the different types and their relative frequencies in the column
def ratiosOfDifferentTypes(column):
    uniquerows = column.drop_duplicates() #.dropna().drop_duplicates() THIS SHOULD BE FIXED!!! WE NEED TO ALSO DO .dropna() !!!
    uniquerows = uniquerows.sample(n=min(1000,uniquerows.size))
    types = [type(entry) for entry in uniquerows]
    differenttypes = list(set(types))
    tally = [types.count(giventype) for giventype in differenttypes]
    total = sum(tally)*1.
    frequencies = [tallyelement / total for tallyelement in tally]
    return (differenttypes,frequencies)

# This function takes a column and decides which type its entries are meant to be like.
# It returns the type. If the entries are so mixed that it can't decide, it returns object.
def decideType(column):
    typesandratios = ratiosOfDifferentTypes(column)
    if max(typesandratios[1]) >= 0.8:
        # all the rows should probably be of the same type and some have been inputted incorrectly
        correcttype = typesandratios[0][typesandratios[1].index(max(typesandratios[1]))]
    else:
        # the rows have a very mixed type and it's not very clear what the correct type is
        correcttype = object
    return correcttype

# This function goes through all columns in the dataframe and returns the name of the columns that are dirty,
# i.e. that have mixed types of entries.
def findMixedTypes(dataframe):
    return [col for col in dataframe if len(ratiosOfDifferentTypes(dataframe[col])[1])>1]

# This function takes a dataframe and for each column says whether it's clean or dirty. If it's dirty,
# it tries to decide which type it should be.
def analyzeColumnTypes(dataframe):
    mixedtypecolumns = findMixedTypes(dataframe)
    if mixedtypecolumns==[]:
        print "All columns have a single type; they are 'clean'. (They may be incorrect though, or have NaNs)."
    else:
        print "The columns",mixedtypecolumns,"have mixed types:\n"
        correcttypes = [(colname,decideType(dataframe[colname])) for colname in mixedtypecolumns]
        for typ in correcttypes:
            if typ[1]==object:
                print " - \'" + typ[0] + "\'" + " is so mixed it's hard to tell the right type"
            else:
                print " - \'" + typ[0] + "\'" + " should be " + "\'" + typ[1].__name__ + "\'"
        print "\nAll other columns have a single type; they are 'clean'. (They may be incorrect though, or have NaNs)."
    return mixedtypecolumns

In [5]:
mixedtypesindataframes = range(len(alldataframes))
for ii in range(len(alldataframes)):
    print "ANALYZING DATAFRAME FROM " + datafilenames[ii] + ":"
    print "----------------------------------------------------"
    mixedtypesindataframes[ii] =  analyzeColumnTypes(alldataframes[ii])
    print "----------------------------------------------------"

ANALYZING DATAFRAME FROM train.csv:
----------------------------------------------------
The columns ['Cabin', 'Embarked'] have mixed types:

 - 'Cabin' should be 'str'
 - 'Embarked' is so mixed it's hard to tell the right type

All other columns have a single type; they are 'clean'. (They may be incorrect though, or have NaNs).
----------------------------------------------------


In [6]:
for ii in range(len(alldataframes)):
    print "ANALYZING DATAFRAME FROM " + datafilenames[ii] + ":"
    print "----------------------------------------------------"
    for col in mixedtypesindataframes[ii]:
        print "Column '" + col + "' has the following unique entries:\n"
        print np.sort(alldataframes[ii][col].unique())
        print

ANALYZING DATAFRAME FROM train.csv:
----------------------------------------------------
Column 'Cabin' has the following unique entries:

[nan 'A10' 'A14' 'A16' 'A19' 'A20' 'A23' 'A24' 'A26' 'A31' 'A32' 'A34'
 'A36' 'A5' 'A6' 'A7' 'B101' 'B102' 'B18' 'B19' 'B20' 'B22' 'B28' 'B3'
 'B30' 'B35' 'B37' 'B38' 'B39' 'B4' 'B41' 'B42' 'B49' 'B5' 'B50'
 'B51 B53 B55' 'B57 B59 B63 B66' 'B58 B60' 'B69' 'B71' 'B73' 'B77' 'B78'
 'B79' 'B80' 'B82 B84' 'B86' 'B94' 'B96 B98' 'C101' 'C103' 'C104' 'C106'
 'C110' 'C111' 'C118' 'C123' 'C124' 'C125' 'C126' 'C128' 'C148' 'C2'
 'C22 C26' 'C23 C25 C27' 'C30' 'C32' 'C45' 'C46' 'C47' 'C49' 'C50' 'C52'
 'C54' 'C62 C64' 'C65' 'C68' 'C7' 'C70' 'C78' 'C82' 'C83' 'C85' 'C86' 'C87'
 'C90' 'C91' 'C92' 'C93' 'C95' 'C99' 'D' 'D10 D12' 'D11' 'D15' 'D17' 'D19'
 'D20' 'D21' 'D26' 'D28' 'D30' 'D33' 'D35' 'D36' 'D37' 'D45' 'D46' 'D47'
 'D48' 'D49' 'D50' 'D56' 'D6' 'D7' 'D9' 'E10' 'E101' 'E12' 'E121' 'E17'
 'E24' 'E25' 'E31' 'E33' 'E34' 'E36' 'E38' 'E40' 'E44' 'E46' 'E49' 'E5

In [27]:
#================================================================================================
# USER INPUT!
# After having run the previous cells you know which columns are clean and dirty.
# Look at the unique values of the dirty ones, given above. Use this information to clean them up

# In each dataframe, there are certain dirty columns that should be numeric
columnsthatshouldbenumeric = [
    ["Survived","Pclass"]
]
# In each column there will be some conventions on how the bad things are written out.
# For each column that should be numeric, we specify a tuple with the info
# (decimaldelimiter (a string),thousanddelimeter (a string), listofstringstoremove (a list))
structureofeachcolumn = [
    [
        (".",",",["-"," ","%"]),
        (".",",",["-"," "])
    ]
]

columnsthatshouldbestrings = [
    ["Name","Sex"]
]

columnsthatshouldbedatetimes = [
    []
]
#================================================================================================

# FROM HERE ON IT'S AUTOMATIC

# This is a test column to see if it all works. CAN BE REMOVED.
# messedupcolumn = pd.Series([1,"200",3,4,5.99,11,"11 , 000 .203","11-2%22"])

# This function takes a column that should be numeric but is all dirty with badly made strings. It removes
# the thousand-delimiters, it replaces the decimaldelimiters with periods, and removes any user-chosen 
# additional set of characters
def turnToNumeric(column,decimaldelimiter=".",thousanddelimeter=",",listofstringstoremove=["-"," ","%"]):
    toremoveregex = str(listofstringstoremove + [thousanddelimeter]).rstrip("]'").lstrip("'[").replace("', '","|")
    numericcolumn = pd.to_numeric(column.astype(str).str.replace(toremoveregex,"").str.replace(decimaldelimiter,"."))
    return numericcolumn

# This function takes a dirty column that should all be strings and turns it into such
def turnToString(column):
    return column.astype(str)

def turnToDate(column):
    return pd.to_datetime(column,dayfirst=True)

for ii in range(len(alldataframes)):
    for (jj,coltofix) in enumerate(columnsthatshouldbestrings[ii]):
        alldataframes[ii].loc[:,coltofix] = turnToString(alldataframes[ii][coltofix])
    
    for (jj,coltofix) in enumerate(columnsthatshouldbedatetimes[ii]):
        alldataframes[ii].loc[:,coltofix] = turnToDate(alldataframes[ii][coltofix])
        
    for (jj,coltofix) in enumerate(columnsthatshouldbenumeric[ii]):
        alldataframes[ii].loc[:,coltofix] = turnToNumeric(alldataframes[ii][coltofix],
                                                    structureofeachcolumn[ii][jj][0],
                                                    structureofeachcolumn[ii][jj][1],
                                                    structureofeachcolumn[ii][jj][2])

In [60]:
#================================================================================================
# USER INPUT!
# Now we need to remove those rows of data that are missing critical information, i.e. remove 
# those rows that have a NaN for something very important.
criticalcolumns = [
    ["Survived","Pclass"]
]
#================================================================================================

# FROM HERE ON IT'S AUTOMATIC

for ii in range(len(alldataframes)):
    alldataframes[ii] = alldataframes[ii].dropna(subset = criticalcolumns[ii])

In [33]:
alldataframes[0].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
